In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.__version__

### MultiLayerPerceptron (MLP) Logistic Regression 

http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html


In [ ]:
# Importing the dataset

import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()
default = pd.read_csv('Default.csv')
default.head()

In [ ]:
X = default.iloc[:,1:].values  
y = default.loc[:,'default']
labelencoder = LabelEncoder()
X[:,0] = labelencoder.fit_transform(X[:,0]) # Encode student
y =  LabelEncoder().fit_transform(default.default) # Encode default
X.shape,y.shape,np.sum(y)

In [ ]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.30, random_state = 1)

# Feature Scaling

sc = MinMaxScaler()
X_train[:,1:] = sc.fit_transform(X_train[:,1:])
X_test[:,1:] = sc.transform(X_test[:,1:])

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Fitting model to the Training set

model = MLPClassifier(solver='lbfgs', activation = 'relu' ,hidden_layer_sizes=(2), random_state=123)
model.fit(X_train, y_train) 

In [ ]:
model.out_activation_,model.n_layers_,model.n_outputs_

In [ ]:
model.coefs_

In [ ]:
print("Score: ",model.score(X_test,y_test))

# Predicting the Test set results
y_pred = model.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cm

#### Use defaults

In [ ]:
model = MLPClassifier(random_state=123) ## Use all defaults
model.fit(X_train, y_train) 
print("Score: ",model.score(X_test,y_test))
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
model.out_activation_,model.n_layers_,model.n_outputs_

In [ ]:
model.get_params()

### MLP for Multinomial Logistic Regression

In [ ]:
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
sns.pairplot(iris,hue='species');

#### Data Preprocessing

In [ ]:
# Covert to arrays
X = iris.iloc[:,0:4].values
y = iris.loc[:,'species'].values

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y,random_state = 1234)

# Scale Data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### Fit Model 

In [ ]:
model = MLPClassifier(solver='lbfgs',activation = 'relu' ,hidden_layer_sizes=(2), random_state=1)

model.fit(X_train, y_train) 

#### Predict Test Set

In [ ]:
print("Score: ",model.score(X_test,y_test))

# Predicting the Test set results
y_pred = model.predict(X_test)

# Making the Confusion Matrix

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
model.out_activation_,model.n_layers_,model.n_outputs_

## Pytorch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
X = default.iloc[:,1:].values  
y = default.loc[:,'default']
labelencoder = LabelEncoder()
X[:,0] = labelencoder.fit_transform(X[:,0]) # Encode student
y =  LabelEncoder().fit_transform(default.default) # Encode degault
X.shape,y.shape,np.sum(y)

In [ ]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.5, random_state = 1)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert data into torch tensors
X_train_t = torch.from_numpy(X_train.astype(np.float32)).to(device)
X_test_t = torch.from_numpy(X_test.astype(np.float32)).to(device)
y_train_t = torch.from_numpy(y_train.astype(np.float32).reshape(-1, 1)).to(device)
y_test_t = torch.from_numpy(y_test.astype(np.float32).reshape(-1, 1)).to(device)

X_train_t.shape, X_test_t.shape, y_train_t.shape, y_test_t.shape

#### Create the classification model class

https://pytorch.org/docs/stable/nn.html

In [ ]:
torch.manual_seed(1)

# Create the classification model class

class LogRegress(nn.Module):
    def __init__(self,num_in,num_out):
        super(LogRegress, self).__init__() 
        self.linear = nn.Linear(num_in, num_out) 
        self.sigmoid = nn.Sigmoid()
  
    def forward(self, x): 
        y_pred = self.linear(x)
        return self.sigmoid(y_pred) 
    
model = LogRegress(X_train.shape[1],1)
model.to(device) # Send to device before specifing optimizer

#### Loss and optimizer

In [ ]:
# Loss and optimizer
criterion = nn.BCELoss() #Binary Cross Entropy
optimizer = torch.optim.Adam(model.parameters())

#### Train the model

In [ ]:
# Train the model
n_epochs = 1000

# Stuff to store
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for it in range(n_epochs):
  
  optimizer.zero_grad() # zero the parameter gradients

  outputs = model(X_train_t) # Forward pass via __call__, outputs is a tensor
    
  loss = criterion(outputs, y_train_t) # Calculate the loss
    
  loss.backward() # Computes the Gradients
  optimizer.step() # Updates the weights

  # Get test loss
  outputs_test = model(X_test_t)
  loss_test = criterion(outputs_test, y_test_t)

  # Save losses
  train_losses[it] = loss.item()
  test_losses[it] = loss_test.item()
    
  if (it + 1) % 50 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {loss_test.item():.4f}')

In [ ]:
# Plot the train loss and test loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();


#### Model prediction and accuracy

In [ ]:

with torch.no_grad():
  p_train = model(X_train_t)  ## Predict
  p_train = np.round(p_train.cpu().numpy())
  train_acc = np.mean(y_train_t.cpu().numpy() == p_train) # Get accuracy

  p_test = model(X_test_t) ## Predict
  p_test = np.round(p_test.cpu().numpy())
  test_acc = np.mean(y_test_t.cpu().numpy() == p_test) # Get accuracy
print(f"Train acc: {train_acc:.4f}, Test acc: {test_acc:.4f}")

In [ ]:
type(y_test),type(y_test_t)

In [ ]:
cm = confusion_matrix(y_test_t.cpu().numpy(), p_test)
cm

In [ ]:
list(model.parameters())

### PyTorch Multinomial

In [ ]:
# Covert to arrays
X = iris.iloc[:,0:4].values
y = iris.loc[:,'species'].values

y =  LabelEncoder().fit_transform(iris.species) # Encode species

In [ ]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 1)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert data into torch tensors
X_train_t = torch.from_numpy(X_train.astype(np.float32)).to(device)
X_test_t = torch.from_numpy(X_test.astype(np.float32)).to(device)
y_train_t = torch.from_numpy(y_train.astype(np.long)).to(device)
y_test_t = torch.from_numpy(y_test.astype(np.long)).to(device)

X_train_t.shape, X_test_t.shape, y_train_t.shape, y_test_t.shape

In [ ]:
torch.manual_seed(1)

# Create the classification model class

class LogRegress(nn.Module):
    def __init__(self,num_in,num_out):
        super(LogRegress, self).__init__() 
        self.linear = nn.Linear(num_in, 7)
        self.relu = nn.ReLU()
        self.out = nn.Linear(7,num_out)
        
        #self.out = nn.Softmax() # no softmax. see CrossEntropyLoss() 
  
    def forward(self, x): 
        z = self.relu(self.linear(x))
        return self.out(z) 
    
model = LogRegress(X_train.shape[1],3)
model.to(device) # Send to device before specifing optimizer

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
y_train_t.shape

In [ ]:
# Train the model
n_epochs = 1000

# Stuff to store
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for it in range(n_epochs):
  
  optimizer.zero_grad() # zero the parameter gradients

  outputs = model(X_train_t) # Forward pass via __call__, outputs is a tensor
    
  loss = criterion(outputs, y_train_t) # Calculate the loss
    
  loss.backward() # Computes the Gradients
  optimizer.step() # Updates the weights

  # Get test loss
  outputs_test = model(X_test_t)
  loss_test = criterion(outputs_test, y_test_t)

  # Save losses
  train_losses[it] = loss.item()
  test_losses[it] = loss_test.item()
    
  if (it + 1) % 50 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {loss_test.item():.4f}')

In [ ]:
# Plot the train loss and test loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();


In [ ]:
def predict(tnsr,dev=False):
    with torch.no_grad():
      logits = model(tnsr)
      if dev: logits.to(dev)
      probs = F.softmax(logits)
      probs = probs.cpu().numpy()
      return list(map(np.argmax,probs))

#### Calculate accuracy

In [ ]:
preds = predict(X_test_t,device)
np.sum(preds == y_test)/len(preds)

In [ ]:
list(model.parameters())

### Exercise

Create and run a PyTorch model classifying wine features to the Customer_Segment Class.  
Do not use a device.  
Use all the features.  
The hidden layer should have 6 nodes  
Run for 500 epochs.  
Calculate the accuracy.  


In [ ]:
wine = pd.read_csv("wine.csv")
wine.tail()